In [132]:
import gzip
from struct import unpack, pack, calcsize
from datetime import timedelta
from collections import deque
import numpy as np

In [133]:
# Offset + length_last - 1
# These values are taken from 
# https://www.nasdaqtrader.com/content/technicalsupport/specifications/dataproducts/NQTVITCHspecification.pdf
message_type_size_dict = {
    b"S" : 11, b"R" : 38, b"H" : 24, b"Y" : 19, b"L" : 25, b"V" : 34, b"W" : 11, b"K" : 27, b"J" : 34, b"h" : 20, b"A" : 35,
    b"F" : 39, b"E" : 30, b"C" : 35, b"X" : 22, b"D" : 18, b"U" : 34, b"P" : 43, b"Q" : 39, b"B" : 18, b"I" : 49, b"N" : 19,
    b"\x00" : 1
}

In [134]:
market_open='0930'
market_close = '1600'

In [135]:
def get_timestamp(binary_value):
    time_stamp = unpack('>6s', binary_value)[0];
    return int.from_bytes(time_stamp, byteorder='big')
    
def handle_system_message():
    global market_o, market_s, market_q, market_m, market_e, market_c
    
    # System Message
    stock_locate, tracking_number = unpack('>HH', msg[0:4])
    event_code = unpack('>c', msg[10:])[0]
    time_stamp = get_timestamp(msg[4:10])
    if event_code == b"O":
        market_o = market_o + 1
        # Start of Messages.Outside oftime stamp messages,the start of day message isthe first message sentin
        # any trading day
    elif event_code == b"S":
        market_s = market_s + 1
        # Start of System hours. This message indicatesthatNASDAQis open and ready to start accepting orders
    elif event_code == b"Q":
        market_q = market_q + 1
        # Start of Market hours. This message isintended to indicate that Market Hours orders are available
        # for execution
    elif event_code == b"M":
        market_m = market_m + 1
        # End of Market hours. This message isintended to indicate that Market Hours orders are no longer
        # available for execution
    elif event_code == b"E":
        market_e = market_e + 1
        # End of System hours. It indicates that Nasdaq is now closed and will not accept any new orderstoday.
        # It is still possible to receive Broken Trade messages and Order Delete messages after the End of Day
    elif event_code == b"C":
        market_c = market_c + 1
        # End of Messages. This is always the last message sent in any trading day.
    converted_time = timedelta(seconds=time_stamp * 1e-9)
    print (stock_locate, tracking_number, converted_time, event_code)

In [158]:
def handle_Add_Order_message(bMPID = False):
    stock = unpack('>8s', msg[23:31])[0]
    time_stamp = get_timestamp(msg[4:10])
    shares = unpack('>I', msg[19:23])[0]
    price = unpack('>I', msg[31:35])[0] * (1e-4)
    converted_time = timedelta(seconds=time_stamp * 1e-9)
    order_ref_number = unpack('>Q', msg[10:18])[0]
    ref_number_stocks.append([order_ref_number, stock])
    

def get_stock(order_ref_number):
    for i in range(len(ref_number_stocks)):
        if ref_number_stocks[i][0] == order_ref_number:
            return (ref_number_stocks[i][1])

def handle_Order_Execute_message():
    printable = unpack('>c', msg[30:31])[0]
    if printable != b'N': # to avoid double counting.
        time_stamp = get_timestamp(msg[4:10])
        shares = unpack('>I', msg[18:22])[0]

        price = unpack('>I', msg[31:])[0] * (1e-4)
        order_ref_number = unpack('>Q', msg[10:18])[0]
        converted_time = timedelta(seconds=time_stamp * 1e-9)
        stock = get_stock(order_ref_number)
        
        match_number = unpack('>Q', msg[22:30])[0]
        
        print ("Execute >>>>>>>>>>.", order_ref_number, stock, converted_time, shares, price, match_number, printable)
        return
        if stock not in stocks_past_cummulative_volume:
            stocks_past_cummulative_volume[stock] = 0
            stocks_past_cummulative_TPV[stock] = 0

        if stock not in stocks_times:
            stocks_times[stock] = deque()
            
        stock_queue = stocks_times[stock]
        stock_queue.append({'time_stamp' : time_stamp,
                            'price' : price})

        while stock_queue[0]['time_stamp'] < (time_stamp - tick_time): 
            stock_queue.popleft()
        
        N = len(stock_queue)
        high = price
        low = price
        for i in range(N):
            if stock_queue[i]['price'] > high:
                high = stock_queue[i]['price']
            if stock_queue[i]['price'] < low:
                low = stock_queue[i]['price']
          
        print (">>>>>>>>>>>>>>>> ", N, high, low, price)
        typical_price = (high + low + price)/3
        current_PV = shares * typical_price
        
        stocks_past_cummulative_TPV[stock] = (stocks_past_cummulative_TPV[stock] +  
                                              current_PV) / (stocks_past_cummulative_volume[stock] + shares)
        stocks_past_cummulative_volume[stock] = shares
        
        print (stock, stocks_past_cummulative_TPV[stock], converted_time)


def handle_trade_NC_message():
    stock = unpack('>8s', msg[23:31])[0]
    price = unpack('>I', msg[31:35])[0] * (1e-4)
    shares = unpack('>I', msg[19:23])[0]
    time_stamp = get_timestamp(msg[4:10])
    converted_time = timedelta(seconds = time_stamp * 1e-9)
    
    order_ref_number = unpack('>Q', msg[10:18])[0]
    match_number = unpack('>Q', msg[35:])[0]
    print ("Trade NC >>>>>>>>>>.", order_ref_number, stock, converted_time, shares, price, match_number)
    return
    if stock not in stocks_times:
            stocks_times[stock] = deque()
    
    stock_queue = stocks_times[stock]
    stock_queue.append({'time_stamp' : time_stamp,
                            'price'  : price,
                            'volumn' : shares})
    
    while stock_queue[0]['time_stamp'] < (time_stamp - tick_time): 
            stock_queue.popleft()

    N = len(stock_queue)
    high = price
    low = price
    prev_cummulative_volumn = 0
    prev_cummulative_PV = 0
    for i in range(N - 1): # don't iterate over current trade
        if stock_queue[i]['price'] > high:
            high = stock_queue[i]['price']
        if stock_queue[i]['price'] < low:
            low = stock_queue[i]['price']
        prev_cummulative_PV = prev_cummulative_PV + stock_queue[i]['typical_price'] * stock_queue[i]['volumn']
        prev_cummulative_volumn = prev_cummulative_volumn + stock_queue[i]['volumn']
        
    print (">>>>>>>>>>>>>>>> ", N, high, low, price)
    typical_price = (high + low + price)/3
    current_PV = shares * typical_price
    stock_queue[-1]['typical_price'] = typical_price
    vwap = (prev_cummulative_PV + current_PV) / (prev_cummulative_volumn + shares)
    
    if stock not in stocks_past_cummulative_volume:
        stocks_past_cummulative_volume[stock] = 0
        stocks_past_cummulative_TPV[stock] = 0
    
    stocks_past_cummulative_volume[stock] = shares
    
    print (">>>>>>>>>>>>>", stock, vwap, converted_time)

In [161]:
#bin_data = open("C:/Users/Amandeep/Downloads/NASDAQ_DATA/01302019.NASDAQ_ITCH50/01302019.NASDAQ_ITCH50", "rb")
bin_data = open("20190130.PSX_ITCH_50", "rb")
message_type = bin_data.read(1);
market_o, market_s, market_q, market_m, market_e, market_c = (0, 0, 0, 0, 0, 0)
number_of_messages = 0
ref_number_stocks = []
stocks_times = {}
stocks_current_avg = {}
stocks_past_cummulative_volume = {}
stocks_past_cummulative_TPV = {}
 # rolling average of Min = 6e10 --- Hour = 3.6e12
tick_time = 6e10
while message_type:
    msg = bin_data.read(message_type_size_dict[message_type])
    number_of_messages += 1
    # trading Message
    if message_type == b"A":
        handle_Add_Order_message()
    elif message_type == b"F":
        handle_Add_Order_message(bMPID = True)
    elif message_type == b"C":
        #pass
        handle_Order_Execute_message()
    elif message_type == b"S":
        handle_system_message()
    elif message_type == b"P":
        handle_trade_NC_message()
    message_type = bin_data.read(1);
    if number_of_messages > 10000000: # for all change to 100000000 or remove this if and break!
        break
    
print (number_of_messages, market_o, market_s, market_q, market_m, market_e, market_c)

0 0 3:06:20.838940 b'O'
0 0 8:00:00.000025 b'S'
Trade NC >>>>>>>>>>. 0 b'UVXY    ' 9:16:35.356124 11 58.95 18427
Trade NC >>>>>>>>>>. 0 b'FB      ' 9:16:37.006757 100 145.82 18431
Trade NC >>>>>>>>>>. 0 b'TRVN    ' 9:25:18.377434 75 1.1 19020
Trade NC >>>>>>>>>>. 0 b'TRVN    ' 9:25:18.378263 925 1.1 19022
Trade NC >>>>>>>>>>. 0 b'IWM     ' 9:25:54.779715 15 146.95000000000002 19057
Trade NC >>>>>>>>>>. 0 b'IWM     ' 9:25:54.852990 30 146.96 19063
Trade NC >>>>>>>>>>. 0 b'BRS     ' 9:28:00.146259 100 3.85 19198
Trade NC >>>>>>>>>>. 0 b'BRS     ' 9:28:05.031185 100 3.85 19207
Trade NC >>>>>>>>>>. 0 b'CFX     ' 9:29:55.124674 100 24.8 19251
0 0 9:30:00.000077 b'Q'
Execute >>>>>>>>>>. 274625 b'EMB     ' 9:30:00.168245 10 107.65 19268 b'Y'
Execute >>>>>>>>>>. 278741 b'XBI     ' 9:30:00.301322 100 80.60000000000001 19277 b'Y'
Execute >>>>>>>>>>. 280091 b'BND     ' 9:30:00.353285 4 79.60000000000001 19282 b'Y'
Trade NC >>>>>>>>>>. 0 b'DWT     ' 9:30:00.441096 200 9.335 19294
Trade NC >>>>>>>>

Execute >>>>>>>>>>. 464395 b'XLE     ' 9:30:57.190141 100 62.980000000000004 20530 b'Y'
Execute >>>>>>>>>>. 465873 b'MU      ' 9:30:58.102819 100 38.580000000000005 20544 b'Y'
Trade NC >>>>>>>>>>. 0 b'CMCSA   ' 9:30:59.832698 100 35.89 20574
Trade NC >>>>>>>>>>. 0 b'SRCI    ' 9:31:00.075205 10 5.13 20583
Trade NC >>>>>>>>>>. 0 b'UNP     ' 9:31:00.090498 100 161.23000000000002 20585
Trade NC >>>>>>>>>>. 0 b'GILD    ' 9:31:00.171508 100 69.74000000000001 20589
Trade NC >>>>>>>>>>. 0 b'CAT     ' 9:31:00.292633 40 127.93 20594
Execute >>>>>>>>>>. 473572 b'XLI     ' 9:31:00.295903 6 71.08 20599 b'Y'
Execute >>>>>>>>>>. 473572 b'XLI     ' 9:31:00.295968 94 71.08 20600 b'Y'
Trade NC >>>>>>>>>>. 0 b'LNT     ' 9:31:00.790019 1 42.79 20609
Execute >>>>>>>>>>. 477488 b'JPM     ' 9:31:01.442059 100 103.9 20636 b'Y'
Trade NC >>>>>>>>>>. 0 b'LLL     ' 9:31:01.594912 12 198.56 20643
Trade NC >>>>>>>>>>. 0 b'AMZN    ' 9:31:01.624640 26 1623.02 20661
Trade NC >>>>>>>>>>. 0 b'GNW     ' 9:31:02.553550 20

Trade NC >>>>>>>>>>. 0 b'BLL     ' 9:31:41.519224 100 51.92 21800
Trade NC >>>>>>>>>>. 0 b'PGTI    ' 9:31:42.922243 100 15.99 21863
Trade NC >>>>>>>>>>. 0 b'AMD     ' 9:31:44.084206 100 21.675 21893
Trade NC >>>>>>>>>>. 0 b'AMD     ' 9:31:45.879889 300 21.645 21924
Trade NC >>>>>>>>>>. 0 b'SHV     ' 9:31:45.899452 10 110.525 21929
Trade NC >>>>>>>>>>. 0 b'AMD     ' 9:31:46.158104 100 21.635 21938
Trade NC >>>>>>>>>>. 0 b'AMD     ' 9:31:46.158642 100 21.625 21939
Trade NC >>>>>>>>>>. 0 b'AMD     ' 9:31:46.302023 100 21.615000000000002 21941
Trade NC >>>>>>>>>>. 0 b'AMD     ' 9:31:46.422822 100 21.605 21943
Trade NC >>>>>>>>>>. 0 b'NVAX    ' 9:31:46.705744 300 2.205 21948
Trade NC >>>>>>>>>>. 0 b'BBT     ' 9:31:47.139350 100 49.190000000000005 21950
Trade NC >>>>>>>>>>. 0 b'PFF     ' 9:31:49.615711 150 35.855000000000004 21998
Trade NC >>>>>>>>>>. 0 b'PFF     ' 9:31:49.616385 100 35.855000000000004 21999
Trade NC >>>>>>>>>>. 0 b'BABA    ' 9:31:50.035873 3 161.12 22006
Trade NC >>>>>>>>>>

Trade NC >>>>>>>>>>. 0 b'PFF     ' 9:32:24.609540 100 35.855000000000004 23097
Trade NC >>>>>>>>>>. 0 b'PFF     ' 9:32:24.610238 100 35.855000000000004 23098
Trade NC >>>>>>>>>>. 0 b'BABA    ' 9:32:24.745500 5 162.52 23106
Trade NC >>>>>>>>>>. 0 b'BABA    ' 9:32:24.745510 5 162.52 23107
Trade NC >>>>>>>>>>. 0 b'BABA    ' 9:32:24.745510 1 162.53 23108
Trade NC >>>>>>>>>>. 0 b'BABA    ' 9:32:24.745549 25 162.53 23109
Trade NC >>>>>>>>>>. 0 b'TMO     ' 9:32:25.056518 100 239.365 23120
Trade NC >>>>>>>>>>. 0 b'AMD     ' 9:32:25.832479 200 21.805 23135
Trade NC >>>>>>>>>>. 0 b'AMD     ' 9:32:25.833005 100 21.805 23136
Trade NC >>>>>>>>>>. 0 b'AMD     ' 9:32:25.833005 200 21.805 23137
Trade NC >>>>>>>>>>. 0 b'RHI     ' 9:32:26.107007 100 66.2 23146
Trade NC >>>>>>>>>>. 0 b'AMZN    ' 9:32:26.209828 3 1624.52 23148
Trade NC >>>>>>>>>>. 0 b'NVDA    ' 9:32:26.566429 59 134.74 23157
Trade NC >>>>>>>>>>. 0 b'AMZN    ' 9:32:26.986710 38 1624.7900000000002 23173
Trade NC >>>>>>>>>>. 0 b'AMD     ' 9:

Trade NC >>>>>>>>>>. 0 b'AMZN    ' 9:33:11.368171 61 1628.8500000000001 24360
Trade NC >>>>>>>>>>. 0 b'AMZN    ' 9:33:11.368171 1 1628.8500000000001 24361
Trade NC >>>>>>>>>>. 0 b'AMZN    ' 9:33:11.368171 1 1628.8500000000001 24362
Trade NC >>>>>>>>>>. 0 b'LBTYK   ' 9:33:11.690964 205 22.92 24371
Execute >>>>>>>>>>. 881808 b'BMY     ' 9:33:11.700346 200 48.74 24372 b'Y'
Trade NC >>>>>>>>>>. 0 b'AMD     ' 9:33:12.025768 100 21.875 24375
Trade NC >>>>>>>>>>. 0 b'AMZN    ' 9:33:12.560951 66 1628.49 24405
Trade NC >>>>>>>>>>. 0 b'UHS     ' 9:33:13.009087 1200 133.05 24420
Trade NC >>>>>>>>>>. 0 b'UHS     ' 9:33:13.011420 70 133.05 24422
Trade NC >>>>>>>>>>. 0 b'CMI     ' 9:33:13.212009 15 146.41 24432
Trade NC >>>>>>>>>>. 0 b'CRM     ' 9:33:13.217097 20 147.57 24434
Trade NC >>>>>>>>>>. 0 b'TCMD    ' 9:33:13.322742 100 65.97 24438
Trade NC >>>>>>>>>>. 0 b'PCG     ' 9:33:13.849691 100 13.74 24454
Trade NC >>>>>>>>>>. 0 b'TEUM    ' 9:33:14.602856 100 2.495 24467
Trade NC >>>>>>>>>>. 0 b'CHDN

...

Execute >>>>>>>>>>. 4687281 b'EEM     ' 9:55:18.835298 100 42.010000000000005 50183 b'Y'
Execute >>>>>>>>>>. 4687281 b'EEM     ' 9:55:18.835298 100 42.010000000000005 50184 b'Y'
Execute >>>>>>>>>>. 4690447 b'PDS     ' 9:55:20.624492 70 2.02 50196 b'Y'
Trade NC >>>>>>>>>>. 0 b'BURL    ' 9:55:24.306820 15 167.98000000000002 50233
Trade NC >>>>>>>>>>. 0 b'NATI    ' 9:55:24.441619 100 41.68 50234
Trade NC >>>>>>>>>>. 0 b'CWB     ' 9:55:24.928608 100 49.605000000000004 50239
Trade NC >>>>>>>>>>. 0 b'CWB     ' 9:55:24.929750 100 49.605000000000004 50240
Trade NC >>>>>>>>>>. 0 b'CLF     ' 9:55:25.219517 100 10.295 50241
Trade NC >>>>>>>>>>. 0 b'CLF     ' 9:55:25.219540 200 10.295 50242
Trade NC >>>>>>>>>>. 0 b'CLF     ' 9:55:25.219791 300 10.295 50243
Trade NC >>>>>>>>>>. 0 b'CLF     ' 9:55:25.220061 400 10.295 50244
Trade NC >>>>>>>>>>. 0 b'CP      ' 9:55:25.523349 100 206.11 50247
Trade NC >>>>>>>>>>. 0 b'AMD     ' 9:55:26.193802 120 22.035 50251
Execute >>>>>>>>>>. 4703726 b'BLMN    ' 9:55

Execute >>>>>>>>>>. 4847662 b'HII     ' 9:56:20.308140 100 204.12 51220 b'Y'
Execute >>>>>>>>>>. 4850311 b'CHK     ' 9:56:21.527484 100 2.85 51225 b'Y'
Execute >>>>>>>>>>. 4850311 b'CHK     ' 9:56:21.530116 900 2.85 51226 b'Y'
Trade NC >>>>>>>>>>. 0 b'APPN    ' 9:56:22.505207 90 31.520000000000003 51244
Trade NC >>>>>>>>>>. 0 b'ADSK    ' 9:56:23.744555 20 141.44 51270
Trade NC >>>>>>>>>>. 0 b'DNR     ' 9:56:26.162455 150 2.025 51298
Trade NC >>>>>>>>>>. 0 b'DNR     ' 9:56:26.162498 50 2.02 51300
Trade NC >>>>>>>>>>. 0 b'ROK     ' 9:56:26.232352 100 170.65 51301
Trade NC >>>>>>>>>>. 0 b'FOXA    ' 9:56:27.070853 100 48.925000000000004 51308
Trade NC >>>>>>>>>>. 0 b'MCD     ' 9:56:27.145467 100 186.69 51311
Trade NC >>>>>>>>>>. 0 b'DE      ' 9:56:27.468113 15 160.41 51313
Trade NC >>>>>>>>>>. 0 b'NVDA    ' 9:56:30.376755 1 132.75 51340
Trade NC >>>>>>>>>>. 0 b'NCLH    ' 9:56:30.509540 28 49.72 51347
Trade NC >>>>>>>>>>. 0 b'DNR     ' 9:56:31.247097 500 2.0100000000000002 51359
Trade NC >>

Execute >>>>>>>>>>. 4976823 b'AMLP    ' 9:57:26.087922 100 9.81 52106 b'Y'
Execute >>>>>>>>>>. 4976823 b'AMLP    ' 9:57:26.087922 60 9.81 52107 b'Y'
Trade NC >>>>>>>>>>. 0 b'ARDX    ' 9:57:27.372275 200 2.12 52122
Trade NC >>>>>>>>>>. 0 b'AMD     ' 9:57:27.375128 100 21.815 52123
Trade NC >>>>>>>>>>. 0 b'AMD     ' 9:57:27.382792 200 21.815 52125
Trade NC >>>>>>>>>>. 0 b'TSLA    ' 9:57:27.389676 5 301.04 52126
Trade NC >>>>>>>>>>. 0 b'TSLA    ' 9:57:27.394148 5 300.84000000000003 52127
Trade NC >>>>>>>>>>. 0 b'TSLA    ' 9:57:27.394333 5 300.81 52128
Trade NC >>>>>>>>>>. 0 b'DFIN    ' 9:57:27.544782 60 14.270000000000001 52133
Trade NC >>>>>>>>>>. 0 b'MLNX    ' 9:57:29.132802 26 85.96000000000001 52177
Trade NC >>>>>>>>>>. 0 b'AMD     ' 9:57:29.468569 100 21.825 52181
Trade NC >>>>>>>>>>. 0 b'FB      ' 9:57:29.503101 100 146.79000000000002 52182
Trade NC >>>>>>>>>>. 0 b'WU      ' 9:57:31.077300 100 18.17 52202
Trade NC >>>>>>>>>>. 0 b'WU      ' 9:57:31.077300 72 18.17 52203
Execute >>>>>

Execute >>>>>>>>>>. 5097779 b'KGC     ' 9:58:15.726498 100 3.27 52878 b'Y'
Execute >>>>>>>>>>. 5111538 b'T       ' 9:58:17.547982 100 29.03 52903 b'Y'
Execute >>>>>>>>>>. 5111538 b'T       ' 9:58:17.548010 100 29.03 52904 b'Y'
Trade NC >>>>>>>>>>. 0 b'TSLA    ' 9:58:17.611877 45 300.98 52910
Execute >>>>>>>>>>. 5112022 b'AMLP    ' 9:58:18.167751 100 9.81 52923 b'Y'
Execute >>>>>>>>>>. 5112022 b'AMLP    ' 9:58:18.167751 60 9.81 52924 b'Y'
Execute >>>>>>>>>>. 5113050 b'BBBY    ' 9:58:18.555529 100 14.4 52931 b'Y'
Execute >>>>>>>>>>. 5113051 b'BBBY    ' 9:58:18.555529 16 14.4 52932 b'Y'
Trade NC >>>>>>>>>>. 0 b'CHK     ' 9:58:18.835012 150 2.85 52934
Trade NC >>>>>>>>>>. 0 b'CHK     ' 9:58:18.835254 150 2.85 52935
Trade NC >>>>>>>>>>. 0 b'CHK     ' 9:58:18.835995 100 2.85 52936
Trade NC >>>>>>>>>>. 0 b'PCG     ' 9:58:19.301256 200 13.64 52938
Trade NC >>>>>>>>>>. 0 b'NHI     ' 9:58:20.580951 85 81.07000000000001 52944
Trade NC >>>>>>>>>>. 0 b'AMD     ' 9:58:23.658852 200 21.675 52967
Trad

Execute >>>>>>>>>>. 5212309 b'NBL     ' 9:59:12.133580 200 22.69 53625 b'Y'
Trade NC >>>>>>>>>>. 0 b'ALRM    ' 9:59:12.571081 100 62.440000000000005 53627
Trade NC >>>>>>>>>>. 0 b'UVXY    ' 9:59:12.648122 15 59.970000000000006 53634
Trade NC >>>>>>>>>>. 0 b'CTL     ' 9:59:12.789384 300 14.515 53642
Trade NC >>>>>>>>>>. 0 b'CTL     ' 9:59:12.789628 500 14.515 53643
Trade NC >>>>>>>>>>. 0 b'AMD     ' 9:59:12.811652 100 21.705000000000002 53645
Execute >>>>>>>>>>. 5216348 b'AJRD    ' 9:59:13.542493 100 38.620000000000005 53658 b'Y'
Trade NC >>>>>>>>>>. 0 b'BEAT    ' 9:59:17.380396 100 68.23 53698
Trade NC >>>>>>>>>>. 0 b'FTNT    ' 9:59:20.243775 100 73.56 53718
Trade NC >>>>>>>>>>. 0 b'PPL     ' 9:59:21.600248 57 30.73 53735
Trade NC >>>>>>>>>>. 0 b'MPC     ' 9:59:22.588324 100 64.41 53738
Trade NC >>>>>>>>>>. 0 b'INFY    ' 9:59:22.603080 100 10.42 53740
Trade NC >>>>>>>>>>. 0 b'CUZ     ' 9:59:23.342181 150 8.805 53741
Trade NC >>>>>>>>>>. 0 b'MO      ' 9:59:24.275112 100 46.09 53756
Trad

In [6]:
calcsize('>HHhL8ccc4c')
calcsize('>HHhLc')
calcsize('HHI')

# Making a 6 byte timestamp
def unpack48(x):
    x1, x2, x3 = struct.unpack('<HHI', x)
    return x1, x2 | (x3 << 16)
calcsize('HH6cc')

11

In [7]:
(1024).to_bytes(2, byteorder='big')

b'\x04\x00'

In [37]:
bin_data = open("20190130.PSX_ITCH_50", "rb")
message_type = bin_data.read(1)
number_of_messages = 0
while message_type:
    msg = bin_data.read(message_type_size_dict[message_type])
    if message_type == b'C':
        handle_Order_Execute_message()
    message_type = bin_data.read(1);
    
    if number_of_messages > 1000000: # for all change to 100000000 or remove this if and break!
        break
    number_of_messages += 1

(274625,) 9:30:00.168245 (10,) 107.65 b'Y'
(278741,) 9:30:00.301322 (100,) 80.60000000000001 b'Y'
(280091,) 9:30:00.353285 (4,) 79.60000000000001 b'Y'
(287675,) 9:30:00.675519 (600,) 40.03 b'Y'
(287675,) 9:30:00.738494 (100,) 40.03 b'Y'
(293554,) 9:30:00.963590 (30,) 146.96 b'Y'
(293554,) 9:30:00.963596 (70,) 146.96 b'Y'
(305869,) 9:30:01.933280 (71,) 101.66000000000001 b'Y'
(280091,) 9:30:05.004557 (49,) 79.60000000000001 b'Y'
(336106,) 9:30:09.676567 (100,) 15.56 b'Y'
(336106,) 9:30:09.684657 (18,) 15.56 b'Y'
(336372,) 9:30:09.741860 (100,) 92.30000000000001 b'Y'
(337011,) 9:30:09.892260 (85,) 43.25 b'Y'
(351246,) 9:30:13.794988 (100,) 111.56 b'Y'
(368162,) 9:30:20.250780 (100,) 63.38 b'Y'
(368164,) 9:30:20.250780 (100,) 63.38 b'Y'
(367067,) 9:30:22.033316 (100,) 53.760000000000005 b'Y'
(382852,) 9:30:26.265546 (100,) 15.790000000000001 b'Y'
(385795,) 9:30:27.340699 (100,) 70.4 b'Y'
(414231,) 9:30:36.885068 (100,) 39.11 b'Y'
(428641,) 9:30:42.980658 (100,) 38.52 b'Y'
(432039,) 9:30:4